In [ ]:
import boto3
import pdfplumber
import io
import csv
import re
import pandas as pd

client = boto3.client('s3');

bucket_name = 'music-industry-data-lake'
obj_key = "raw/pollstar_dir/pdf/reports_pack/top-tours/2019/north-american/Top200NorthAmericanTours_791.pdf"

non_musician_tours = []
non_musicians_path = "../data_cleaning/non_musicians.txt"

with open(non_musicians_path, 'r') as file:
    for line in file:
        non_musician_tours.append(line.strip())

try:
    file = client.get_object(Bucket=bucket_name, Key=obj_key)
    pdf_bytes = file['Body'].read()
    pdf_file = io.BytesIO(pdf_bytes)

    with pdfplumber.open(pdf_file) as pdf:
        top_tours_musicians = []
        top_tours_all = []

        for page in pdf.pages:
            page_text = page.extract_text()
            print(page_text)
            lines = page_text.splitlines()

            for line in lines:
                #print(line)
                if re.search(r"^\d", line):
                    try:
                        tour_data = re.split(r"\s", line, 2);
                        rank = tour_data[0]
                        gross_millions = tour_data[1]
                        tour_data = tour_data[2]
                        #print(tour_data)

                        end_of_artist_name_idx = re.search(r"\d*\.\d*", tour_data)
                        artist = tour_data[0:end_of_artist_name_idx.start()-2]
                        print(artist)
                        tour_data = tour_data[end_of_artist_name_idx.start():]
                        print(tour_data)

                        view_remaining_columns = re.split(r"\s", tour_data, 5)
                        #print(view_remaining_columns)

                        # if its a full detail table
                        if "/" in view_remaining_columns[4]:
                            tour_data = re.split(r"\s", tour_data, 5)
                            avg_ticket_price = tour_data[0]
                            avg_num_tickets = tour_data[1]
                            total_num_tickets = tour_data[2]
                            avg_gross = tour_data[3]
                            cities_shows = tour_data[4]
                            agency = tour_data[5]

                            tour_dict = {
                            "Rank": rank,
                            "Gross Millions": gross_millions,
                            "Artist": artist,
                            "Average Ticket Price": avg_ticket_price,
                            "Average Tickets": avg_num_tickets,
                            "Total Tickets": total_num_tickets,
                            "Average Gross": avg_gross,
                            "Cities Shows": cities_shows,
                            "Agency": agency
                            }

                            top_tours_all.append(tour_dict)

                            if artist not in non_musician_tours:
                                top_tours_musicians.append(tour_dict)

                        # otherwise its an abridged table
                        else:
                            tour_data = re.split(r"\s", tour_data, 5)
                            avg_ticket_price = tour_data[0]
                            avg_num_tickets = tour_data[1]
                            avg_gross = tour_data[2]
                            rank2 = tour_data[3]
                            gross_millions2 = tour_data[4]

                            tour_data = tour_data[5]
                            end_of_artist_name_idx = re.search(r"\d*\.\d*", tour_data)
                            artist2 = tour_data[0:end_of_artist_name_idx.start()-2]
                            tour_data = tour_data[end_of_artist_name_idx.start():]
                            print(tour_data)
                            tour_data = re.split(r"\s", tour_data)

                            avg_ticket_price2 = tour_data[0]
                            avg_num_tickets2 = tour_data[1]
                            avg_gross2 = tour_data[2]

                            tour_dict1 = {
                                "Rank": rank,
                                "Gross Millions": gross_millions,
                                "Artist": artist,
                                "Average Ticket Price": avg_ticket_price,
                                "Average Tickets": avg_num_tickets,
                                "Total Tickets": None,
                                "Average Gross": avg_gross,
                                "Cities Shows": None,
                                "Agency": None
                            }

                            tour_dict2 = {
                                "Rank": rank2,
                                "Gross Millions": gross_millions2,
                                "Artist": artist2,
                                "Average Ticket Price": avg_ticket_price2,
                                "Average Tickets": avg_num_tickets2,
                                "Total Tickets": None,
                                "Average Gross": avg_gross2,
                                "Cities Shows": None,
                                "Agency": None
                            }
                            
                            top_tours_all.append(tour_dict1)
                            top_tours_all.append(tour_dict2)

                            if artist not in non_musician_tours:
                                top_tours_musicians.append(tour_dict1)

                            if artist2 not in non_musician_tours:
                                top_tours_musicians.append(tour_dict2)
                                            
                    except Exception as e:
                        print("Error parsing tour record: ", e)

        #print(top_tours_all)
        #df_all_tours = pd.DataFrame(top_tours_all)
        #df_musician_tours = pd.DataFrame(top_tours_musicians)
    
        #df_all_tours.to_csv("./csv/Top200Tours-AllTours.csv")          
        #df_musician_tours.to_csv("./csv/Top200Tours-Musicians.csv")
                        

except client.exceptions.NoSuchKey:
    print("Object not found in bucket")
except Exception as e:
    print(f"An error occurred: {e}")
